# 使用 Transformer 进行文本分类
https://keras.io/examples/nlp/text_classification_with_transformer/

In [23]:
import keras
from keras import ops
from keras import layers

## 将 Transformer 块实现为层
参数：embed_dim: 输入向量的维度(嵌入维度)，num_heads: 多头注意力机制的头数，ff_dim: 前馈神经网络中间层的维度，rate: dropout率，默认为0.1<br><br>
组件:self.att: 多头注意力层(Multi-Head Attention)。self.ffn: 前馈神经网络(Feed Forward Network)，两个全连接层。self.layernorm1 和 self.layernorm2: 两个层归一化(Layer Normalization)。self.dropout1 和 self.dropout2: 两个dropout层<br><br>
实现遵循了原始Transformer论文的标准结构：多头自注意力层，残差连接 + 层归一化，前馈神经网络，残差连接 + 层归一化<br><br>
这种结构使得模型能够:通过自注意力机制捕获序列中元素间的长距离依赖关系，通过残差连接缓解深层网络的梯度消失问题，通过层归一化稳定训练过程，通过dropout防止过拟合<br><br>
<strong>ffn</strong> 是 Feed-Forward Network（前馈神经网络）的缩写，ffn 通常是一个 两层的全连接神经网络，用于对自注意力层（MultiHeadAttention）的输出进行非线性变换，增强模型的表达能力。

In [28]:
# 实现了Transformer块（编码器架构）
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        # 多头自注意力
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # 层归一化
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        # 前馈神经网络
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        # 层归一化
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        # 丢弃层
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    # 前向传播方法
    def call(self, inputs):
        # 自注意力机制
        attn_output = self.att(inputs, inputs)  # 计算输入序列的自注意力
        attn_output = self.dropout1(attn_output)  # 正则化
        out1 = self.layernorm1(inputs + attn_output)  # 第一次残差连接和层归一化

        # 前馈神经网络
        ffn_output = self.ffn(out1) # 通过一个两层的ffn处理数据
        ffn_output = self.dropout2(ffn_output)  # 正则化
        return self.layernorm2(out1 + ffn_output)  # 残差连接层归一化

## 实现嵌入层
两个单独的嵌入层，一个用于token，一个用于token索引（位置）。<br><br>
参数：maxlen: 序列的最大长度，vocab_size: 词汇表大小，embed_dim: 嵌入维度<br><br>
初始化了两个嵌入层：token_emb: 用于 token 的嵌入，将词汇 ID 映射为向量。pos_emb: 用于位置的嵌入，将位置索引映射为向量

In [31]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        # 获取输入序列的实际长度
        maxlen = ops.shape(x)[-1]
        # 生成位置序列 [0, 1, 2, ..., maxlen-1]
        positions = ops.arange(start=0, stop=maxlen, step=1)
        # 获取位置嵌入
        positions = self.pos_emb(positions)
        # 获取 token 嵌入
        x = self.token_emb(x)
        # 将两者相加作为最终输出
        return x + positions

## 下载并准备数据集

In [33]:
# 方案一 IMDB 电影评论数据集（IMDB Movie Reviews Sentiment Dataset）
# 它是 Keras 内置的一个经典 NLP 数据集，包含 50,000 条带有正面/负面标签的影评，常用于文本分类任务。

vocab_size = 50000  # Only consider the top 20k words
maxlen = 300  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

25000 Training sequences
25000 Validation sequences


In [32]:
# 方案二 Amazon Reviews
# 数据量：数千万条商品评论（比 IMDB 大得多）。
# 获取方式：通过 Amazon Product Data 或 Hugging Face 的 amazon_polarity 数据集。

from datasets import load_dataset
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 加载数据集
dataset = load_dataset("amazon_polarity")
train_data, val_data = dataset["train"], dataset["test"]

# 文本编码（需先构建词汇表，或用 Hugging Face Tokenizer）
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(train_data["content"])
x_train = tokenizer.texts_to_sequences(train_data["content"])
x_val = tokenizer.texts_to_sequences(val_data["content"])

# 填充序列
maxlen = 200
x_train = pad_sequences(x_train, maxlen=maxlen)
x_val = pad_sequences(x_val, maxlen=maxlen)

# 标签
y_train = train_data["label"]
y_val = val_data["label"]
print("Unique labels:", np.unique(y_train))  # 应输出 [0, 1]

KeyboardInterrupt: 

## 使用 transformer 层创建分类器模型
Transformer 层为输入序列的每个时间步输出一个向量。 在这里，我们取所有时间步长的均值，并且 在其上使用前馈网络对文本进行分类。<br><br>
参数：embed_dim: 控制嵌入向量的大小。num_heads: 决定多头注意力的并行注意力机制数量。ff_dim: Transformer内部前馈网络的维度<br><br>

In [34]:
embed_dim = 32  # 每个token的嵌入维度
num_heads = 2  # 注意力头的数量
ff_dim = 32  # Transformer前馈网络的隐藏层大小

# 输入层：定义模型输入，形状为 (maxlen,)，即固定长度的序列
inputs = layers.Input(shape=(maxlen,))

# 嵌入层
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)  # 使用之前定义的 TokenAndPositionEmbedding 层
x = embedding_layer(inputs)  # 将输入的 token IDs 转换为嵌入向量并添加位置信息

#  Transformer 块
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)  # 应用 Transformer 块处理序列数据
x = transformer_block(x) 

# 全局池化
x = layers.GlobalAveragePooling1D()(x)  # 对序列维度进行平均池化，将变长序列转换为固定长度的表示（维度=embed_dim）

# 分类头部
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

# 模型定义
model = keras.Model(inputs=inputs, outputs=outputs)

## 训练和评估

In [35]:
# 方案一
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
)


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 88s 110ms/step - accuracy: 0.6649 - loss: 0.5586 - val_accuracy: 0.8840 - val_loss: 0.2817
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 79s 101ms/step - accuracy: 0.9425 - loss: 0.1667 - val_accuracy: 0.8803 - val_loss: 0.3039
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 92ms/step - accuracy: 0.9707 - loss: 0.0886 - val_accuracy: 0.8683 - val_loss: 0.4040
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 90ms/step - accuracy: 0.9882 - loss: 0.0449 - val_accuracy: 0.8510 - val_loss: 0.6051
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 76s 97ms/step - accuracy: 0.9912 - loss: 0.0307 - val_accuracy: 0.8486 - val_loss: 0.7572


In [13]:
# 方案二 使用Amazon Review数据集
# 编译模型
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # 适用于整数标签
    metrics=["accuracy"]
)

# 训练模型
history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=2,
    validation_data=(x_val, y_val)
)

# 评估模型
loss, accuracy = model.evaluate(x_val, y_val)
print(f"Validation Accuracy: {accuracy:.4f}")

ValueError: Unrecognized data type: x=[[   0    0    0 ... 2730    5  329]
 [   0    0    0 ...  174  158 2700]
 [   0    0    0 ...    6  402  266]
 ...
 [   0    0    0 ...   45  690 1255]
 [   0    0    0 ...    7    1   35]
 [   0    0    0 ...   18 4513 2101]] (of type <class 'numpy.ndarray'>)